# SAAS <> Media Matters Worldwide Media Mix Model and Budget Optimizer

This notebook contains a media mix model, created from methods similar to __[this paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46001.pdf)__. The model informs a budget optimizer, which optimizes the budget for spending in advertising media channels in the short-term.

There are 4 sections in this notebook.

1. [Data Collection](#data-collection)
2. [Data Preprocessing](#data-preprocessing)
3. [Modeling](#modeling)
4. [Optimization](#optimize)

> Run a notebook-cell by clicking a cell and pressing ctrl-enter (windows) or  (equivalent on mac). Alternatively, after selecting a cell, hit the play button in the navigation bar at the top

> Don't modify the code in any cell unless explicitly prompted to do so.     

***

Replace the `client` variable with the client's name. The client's name should be surrounded by double quotations.

In [ ]:
client = "MHW"

Run the following three cells to import the necessary packages

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score 
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

from datetime import datetime as dt
from functools import reduce
from yellowbrick.regressor import residuals_plot
from transformations import transform_one_column, select_best_feature, feature_imp_random_forest

import time
import datetime
import ipywidgets as widgets
from ipywidgets import FileUpload

from IPython.display import display
from IPython.display import FileLink
import io
import re
from scipy.optimize import minimize, LinearConstraint

import openpyxl
import xlrd

import holidays

import panel as pn
pn.extension()

In [ ]:
from v4_helper import *

In [ ]:
# COMMANDS I RAN TO GET FILEUPLOAD AND JUPYTERWIDGETS TO LOAD
# !jupyter nbextension enable --py widgetsnbextension
# !pip install jupyterlab-widgets
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# uf = upload.value[list(upload.value.keys())[0]]['content']

***

<a id="data-collection"></a> 
# Data Collection

<div class="alert alert-block alert-info">
<b>Note: </b>The notebook is only compatible with two file formats: .csv and .xlsx.</div>

## 1A: Input Data - Sales

Set the `use_excel` variable in the next cell to `False` if you are inputting a `.csv` file; if you are inputting a `xlsx`, leave as `True` .
Set the `granularity` variable in the next cell to `False` if the granularity (what each row in the data represents) is weekly; if daily, leave as `True`.
> The granularity of the sales data must be the same as the granularity of the advertising mix data

In [ ]:
use_excel = True
granularity = True
data_type = "SALES DATA"

***

### step 1: attach file

The next cell will take you through the steps to input the data. Don't modify any of the parameters (`data_type` and `use_excel`) in the cell.
After running the cell, an *Upload* Button will pop up. Click the *Upload* Button, navigate to the folder on your computer that contains the sales data, and attach the proper file. Once you upload the *sales data*, the number next to *Upload* should change from 0 to 1. If so, proceed by running the next cell.

> The *Upload* widget will only recognize and look for .csv or .xlsx formats.

In [ ]:
df_as_bytes = input_file(data_type, use_excel=use_excel)

***

### step 2: parse sales data

Now that you've attached the sales data, the notebook will need to parse the data from the *Upload* Button. 
After running the cell, the notebook will display the sales data in a table. 
The following cell will ask you the following questions regarding the format of the data.

1. Is the header, as displayed, of the dataframe correct? [y/n] If no, which row contains the header (zero-indexed)?
2. Is the SALES DATA file good to go? [y/n]

`y` corresponds to yes, and `n` corresponds to no. `zero-indexed` is a way of numbering in which the initial element of a sequence is assigned the index 0, rather than 1.

In [ ]:
sales_df = parse_file(df_as_bytes, data_type, use_excel=use_excel)

***

### step 3: clean sales data

After running the cell, the notebook will display a list of the column-names of the sales data.

This cell asks you to identify the different countries or regions represented in the data, identify which column corresponds to the (day/week) and identify which column corresponds to the target or dependent variable (revenue/orders).

> Type each column name exactly as it is displayed, without quotations

In [ ]:
sales_df_cleaned = clean_sales(sales_df, data_type, granularity=granularity)

***

This process will be repeated for the advertising media mix data.

## 1B: Input Data - Ad Mix

Now that you've uploaded the sales data, the notebook requires advertising mix data.
Set the `use_excel` variable in the next cell to `False` if you are inputting a .csv file; if not, leave as `True`.

> The granularity of this data must be the same granularity of the sales data.

In [ ]:
use_excel = True

***

### step 1: input ad data

In [ ]:
data_type = "AD DATA"

In [ ]:
ad_df_as_bytes = input_file(data_type, use_excel=use_excel)

***

### step 2: parse ad data

Now that you've attached the ad data, like before, the notebook will need to parse the data from the *Upload* Button. 
After running the cell, the notebook will display the ad data in a table. 
The following questions regarding the format of the data will be asked.

1. Is the header, as displayed, of the dataframe correct? [y/n] If no, which row contains the header (zero-indexed)?
2. Is the ad mix data file good to go? [y/n]

`y` corresponds to yes, and `n` corresponds to no. `zero-indexed` is a way of numbering in which the initial element of a sequence is assigned the index 0, rather than 1.

In [ ]:
ad_df = parse_file(ad_df_as_bytes, data_type, use_excel=use_excel)

***

### step 3: clean ad mix data and merge with sales data

After running the cell, the notebook will display a list of the column-names of the ad mix data. This cell asks you to identify the column in the ad data corresponding to the country/region, media cost, and impressions.

The output of this cell, `data_dict`, is a dictionary mapping country name to its corresponding data matrix.

In [ ]:
data_dict = clean_and_merge(ad_df, sales_df_cleaned)

***

<a id="data-preprocessing"></a> 
# 2) Data Preprocessing

This cell finds the optimal parameters for the `carryover` and `diminishing return` transformations. 

The notebook is only capable of working with one dataset at a time. Run the next cell to get a list of valid country names. Set the value of `country` in the next cell to one of the values in the list of country names. Ensure the country name is surrounding by double quotations, like `"country_name"`. 

> The results from the remainder of the notebook will be only for this country.

In [ ]:
possible_countries = data_dict.keys()
print("Possible Countries to choose from", list(possible_countries))

Set the `target` variable to the dependent variable, `Revenue` or `Orders`, or something else.

In [ ]:
country = "US"
target = 'Orders'

> This cell takes between two and 15 minutes to run

In [ ]:
data_matrix = apply_transformations(data_dict, country=country, response_var=target)

***

<a id="modeling"></a> 
# 3) Modeling

The model, ridge regression, will be trained. The model can be set to `additive` or `multiplicative`, which makes another set of assumptions. Set the `model_type` variable below to either `"additive"` or `"multiplicative"`, maintaining the double quotations. Set the `include_impressions` variable to `True` if you'd like to include impressions in the model (at the risk of overfitting), or leave as `False`.

## model fitting

In [ ]:
model_type = "additive"
include_impressions = False

In [ ]:
if model_type == "additive":
    x, y = prep_additive_model(data_matrix.fillna(0), target)
    if not include_impressions:
        x = x[get_media_vars(x)]
    model = ridge_regression(x, y)
elif model_type == "multiplicative":
    x, y = prep_mult_model(data_matrix.fillna(0), target)
    if not include_impressions:
        x = x[get_media_vars(x)]
    model = ridge_regression(x, y)

***

## model evaluation

Here's a graph of how well the model performs on the training set.
> MAPE = mean absolute percent error between the true revenue and predicted revenue (from trained model)

In [ ]:
pred_vs_true(model, x, y)

### one-day forecasting

In [ ]:
box, c = init_sliders(model)

In [ ]:
box

Run the next cell. Verify that the specified slider values match the output.

In [ ]:
get_slider_vals(model, c)

In [ ]:
if model_type == "additive":
    arr =  add_predict(data_dict[country].copy(), model, c)
elif model_type == "multiplicative":
    arr =  mult_predict(data_dict[country].copy(), model, c)

### One-Month Forecasting

Run the next cell and proceed.

In [ ]:
get_media_vars(x)

To forecast for one-month (or any time length greater than 1 day), write an excel spreadsheet or .csv file with the above columns in that **EXACT** order. Set `use_excel` to `True` if the file is an excel file, else set it to `False`.

In [ ]:
data_type = "ad data [forecasting]"
use_excel = True

In [ ]:
forecast_as_bytes = input_file(data_type, use_excel=use_excel)

In [ ]:
forecast_sales_df = parse_file(forecast_as_bytes, data_type, use_excel=use_excel)

In [ ]:
forecast_output_df = month_predict(forecast_sales_df.copy(), data_dict[country].copy(), model, model_type)

#### plot of predictions

Run the next cell to get a plot of predicted revenue for one month (or more) of forecasting. To export these predictions, see the bottom of this notebook for export options.

In [ ]:
plt.plot(forecast_output_df.predictions)
plt.title("Forecasted Revenue");

***

<a id="optimize"></a> 
# 4) Optimization

We can use the training ridge regression model to optimize the mix. The optimizer can optimize the budget for a single day/week, and for a month. Run the next code cell to get a list of range sliders. Each slider can be adjusted accordingly to set the lower and upper bounds for each column. Then, run the rest of the cells. 

> Sometimes, the optimizer won't converge, due to either lack of data or lack of statistical significance.

## model for optimization

In [ ]:
x, y = prep_additive_model(data_matrix.fillna(0), target)
x_opt = x[get_media_vars(x)]
model_opt = ridge_regression(x_opt, y)

## optimized budget - day

Run the next cell and set the lower and upper bounds appropriately by moving the cursors on the end. Unfortunately, there is no way to type values in as bounds, so be precise when setting the bounds.

> The one-day optimizer depends heavily on the past two weeks of data, so the one-day optimizer sometimes struggles to find meaningful results. This is corrected by the one-month+ optimizer.

In [ ]:
dm2 = pd.concat([x_opt, y], axis=1)
bounds, a = set_bounds(model_opt)
bounds

In the next cell, edit the budget parameter by setting the value after `budget=` to whatever specified budget size you'd like. Then, run the cell.

In [ ]:
b = add_optimize_budget(data_dict[country].copy(), dm2.copy(), model_opt, a, budget=30000)

In [ ]:
graph(b, 'optimized budget')

***

## optimized budget - month

To get the optimized budget for a month, run the next cell to get more bounds sliders. After that, run the next two cells, which will output a graph. To export the optimized budget to an excel spreadsheet, run the code under this section.

If the graph has low variance and values converge, the columns for which the spend converges are yet to be saturated. If the graph has high variance and values don't converge, under our assumptions, spend for the non-converging channels has reached saturation.

In [ ]:
bounds

In [ ]:
llb = add_optimize_month(data_dict[country].copy(), data_matrix.copy(), model_opt, col_to_slider=a, budget=3000)

In [ ]:
graph_month(llb, x_opt)

***
## Exports

### One-Month Forecasting

Add a unique file name into the string in the next cell, leaving out the file suffix. Then, run the cell after that to get a download link to the forecast. The Excel spreadsheet will have a new column containing the predictions.

In [ ]:
unique_file_name = "SAAS_one_month_fcastA"

In [ ]:
def export_month_forecasting(forecast_output_df, client, unique_file_name):
    forecast_output_df.to_excel(unique_file_name + ".xlsx", index=False)
    file = FileLink(unique_file_name + ".xlsx")
    display(file)
export_month_forecasting(forecast_output_df, client, unique_file_name)

### Optimized Month

Run this cell to get a download link to the optimized budget. This cell requires the optimized_budget - month section to be run.

In [ ]:
def export_optimized_month(llb, x_opt, client):
    name = f"{client}_optimized_month.xlsx"
    pd.DataFrame(columns=[shorten_f_name(y) for y in x_opt.columns], data=np.array(llb)[:, :len(x_opt.columns)]).to_excel(name, index=False)
    file = FileLink(name)
    display(file)
export_optimized_month(llb, x_opt, client)

### Model Coefficients

Run this cell to get a download link to the model's coefficients.

In [ ]:
def export_model_coefficients(model, client):
    name = f"{client}_model_coefficients.xlsx"
    model_coefficients = dict(zip([shorten_f_name(y) for y in model.feature_names_in_], model.coef_))
    pd.DataFrame(data=model_coefficients, index=[0]).to_excel(name, index=False)
    file = FileLink(name)
    display(file)

export_model_coefficients(model, client)

This notebook was designed by __[SAAS Berkeley Data Consulting](https://saas.berkeley.edu/)__